In [ ]:
from transformers import pipeline
EN_MODEL = "jonatasgrosman/wav2vec2-large-xlsr-53-english"

pipe_en = pipeline(model=EN_MODEL, device=0)

In [1]:
sentences = [
	"So, we decided to go to the zoo, and... uh, the first thing we saw was the lions. [laughs] They were just lazing around.",
	"[gasps] Oh, and you won't believe the size of the elephants. They were HUGE.",
	"We were walking past the monkey enclosure when one of them [laughs]—seriously, it started mimicking us!",
	"[sighs] The weather was kinda hot, but... it was worth it, seeing all those animals up close.",
	"And the penguins, they were so cute, waddling around. [laughs] I could watch them all day.",
	"I was a bit scared of the snakes, not gonna lie. Every time one moved, I just... [gasps] jumped a little.",
	"[clears throat] Oh, and the food there? Surprisingly good. We had these amazing burgers right by the giraffe area.",
	"The kids were just fascinated by the aquarium section. All those colorful fish and... uh, the sharks were a bit scary, though.",
	"[laughter] You should've seen your face when that parrot started talking! Priceless.",
	"And... um, at the end of the day, we were all just so tired, but, you know, it was a great day. [sighs] Really special."
]

In [ ]:
from pathlib import Path

outputs = []
files = []

for file in Path("/home/joregan/shivam-sentences").glob("*.wav"):
    files.append(file.name)
    en_out = pipe_en(str(file), chunk_length_s=10, return_timestamps="word")
    outputs.append(en_out)

In [25]:
from string import punctuation

PUNCT = set(punctuation)

def clean_sentence(text):
    words = []
    text = text.replace("—", " ")
    for word in text.split(" "):
        if word.startswith("[") and word.endswith("]"):
            continue
        while word[0:1] in PUNCT:
            word = word[1:]
        while word[-1:] in PUNCT:
            word = word[:-1]
        words.append(word.lower())
    return " ".join(words)

In [26]:
clean_sentences = [clean_sentence(x) for x in sentences]

In [ ]:
numbers = [int(x.replace(".wav", "")) for x in files]
text_out = [x["text"] for x in outputs]
tmp_sort = sorted([x for x in zip(numbers, text_out)])
hyp = [x[1] for x in tmp_sort]

In [ ]:
from jiwer import wer

wer(clean_sentences, hyp)

```python
>>> wer(clean_sentences, hyp)
0.40782122905027934
```